In [1]:
import csv
import mysql.connector

In [2]:
# Thực hiện kết nối tới cơ sở dữ liệu MySQL
config = {
        'user': 'root',
        'password': 'debezium',
        'host': '127.0.0.1',
        'port': '3306',
        'database': 'myCompany'
    }
connection = mysql.connector.connect(**config)

In [3]:
dict_list_order = list()
with open('./data/order.csv', mode='r') as order_csv:
    order_reader = csv.reader(order_csv)
    header = next(order_reader)
    for rows in order_reader:
        dict_list_order.append({'product_id':rows[0], 'quantity':rows[1], 'created_at':rows[2]})

dict_list_odetail = list()
with open('./data/order_detail.csv', mode='r') as odetail_csv:
    odetail_reader = csv.reader(odetail_csv)
    header = next(odetail_reader)
    for rows in odetail_reader:
        dict_list_odetail.append({'order_id':rows[0], 'user_id':rows[1], 'total':rows[2], 'payment':rows[3]})

        
# Tạo đối tượng cursor để thực hiện truy vấn SQL
mycursor = connection.cursor()
for order_item, odetail_item in zip(dict_list_order, dict_list_odetail):
    sql1 = "INSERT INTO Orders(product_id, quantity, created_at) VALUES (%s, %s, %s)"
    val1 = order_item['product_id'], order_item['quantity'], order_item['created_at']
    mycursor.execute(sql1, val1)
    sql2 = "INSERT INTO Order_detail(order_id, user_id, total, payment) VALUES (%s, %s, %s, %s)"
    val2 = odetail_item['order_id'], odetail_item['user_id'], odetail_item['total'], odetail_item['payment']
    mycursor.execute(sql2, val2)
connection.commit()
connection.close()